In [22]:
# 필요한 Import
import pandas as pd
from PIL import Image
import os
from tqdm import tqdm 
from concurrent.futures import ProcessPoolExecutor, as_completed

In [ ]:
# --- 경로 설정 ---
data_dir = 'safebooru\data'
tmp_dir = os.path.join(data_dir, 'tmp') # 원본 이미지 다운로드 폴더
process_image_dir = os.path.join(data_dir, 'images') # 전처리된 이미지 저장 경로
data_path = os.path.join(data_dir, 'all_data.csv')

In [24]:
# --- 전처리 설정 ---
MIN_SIZE = 224
MAX_SIZE = 10000
OUTPUT_SIZE = 224

In [25]:
# CPU 집중 작업이므로 코어 수에 맞춰 설정
WORKERS = os.cpu_count() - 2  # 시스템 부하를 위해 2코어 여유

print(f"✅ 설정 완료")
print(f"전처리 워커 수: {WORKERS}")

✅ 설정 완료
전처리 워커 수: 14


In [26]:
def resize_with_padding(image, size):
    image.thumbnail((size, size), Image.Resampling.LANCZOS)
    new_image = Image.new('RGB', (size, size), (0, 0, 0))
    offset = ((size - image.width) // 2, (size - image.height) // 2)
    new_image.paste(image, offset)
    return new_image

def process_single_image(img_path, output_path):
    try:
        # UserWarning을 일시적으로 무시하여 깔끔한 출력을 유도할 수 있습니다.
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", UserWarning)
            img = Image.open(img_path).convert('RGB')
        
        processed_img = resize_with_padding(img, OUTPUT_SIZE)
        processed_img.save(output_path, quality=95)
        # 성공 시 (경로, "success") 튜플 반환
        return (img_path, "success")
    except Exception as e:
        # 실패 시 (경로, "에러 메시지") 튜플 반환
        return (img_path, f"error: {e}")

# Image, warnings 라이브러리가 필요하므로 셀 상단에 추가해야 할 수 있습니다.
import warnings
from PIL import Image

print("✅ 헬퍼 함수 정의 완료")

✅ 헬퍼 함수 정의 완료


In [27]:
# 1. 메타데이터 로드 및 유효 ID 생성
print("--- 1. 메타데이터 로딩 ---")
df = pd.read_csv(data_path)
valid_df = df[(df['sample_width'] >= MIN_SIZE) & (df['sample_height'] >= MIN_SIZE) & 
              (df['sample_width'] <= MAX_SIZE) & (df['sample_height'] <= MAX_SIZE)]
valid_ids = set(valid_df['id'].astype(str))
print(f"-> CSV 기준, 유효한 이미지 ID 수: {len(valid_ids)}개")

# 2. 전처리할 작업 목록 생성 및 진단
print("\n--- 2. 처리 대상 파일 탐색 ---")
tasks = []
already_processed_count = 0
invalid_id_count = 0
source_file_count = 0

try:
    image_files = os.listdir(tmp_dir)
    source_file_count = len(image_files)
    print(f"-> 소스 폴더(tmp)에서 찾은 파일 수: {source_file_count}개")
    
    for img_name in image_files:
        img_id = os.path.splitext(img_name)[0]
        
        # ID 유효성 검사
        if img_id not in valid_ids:
            invalid_id_count += 1
            continue
        
        # 결과물 존재 여부 검사
        output_path = os.path.join(process_image_dir, f"{img_id}.jpg")
        if os.path.exists(output_path):
            already_processed_count += 1
            continue
            
        # 모든 조건을 통과하면 작업 목록에 추가
        img_path = os.path.join(tmp_dir, img_name)
        tasks.append((img_path, output_path))
        
except FileNotFoundError:
    print(f"오류: 소스 폴더 '{tmp_dir}'를 찾을 수 없습니다. 경로를 확인해주세요.")

# 3. 최종 진단 결과 출력
print("\n--- 3. 진단 결과 ---")
print(f"CSV에 없는 ID를 가진 파일 수: {invalid_id_count}개")
print(f"이미 처리된 파일 수: {already_processed_count}개")
print(f"🔥 최종적으로 전처리할 이미지 수: {len(tasks)}개")

--- 1. 메타데이터 로딩 ---
-> CSV 기준, 유효한 이미지 ID 수: 3014341개

--- 2. 처리 대상 파일 탐색 ---
-> 소스 폴더(tmp)에서 찾은 파일 수: 15680개

--- 3. 진단 결과 ---
CSV에 없는 ID를 가진 파일 수: 0개
이미 처리된 파일 수: 0개
🔥 최종적으로 전처리할 이미지 수: 15680개


In [29]:
# 병렬 처리 대신, 안정적인 일반 for문으로 순차 처리합니다.
if tasks: # 처리할 작업이 있을 때만 실행
    print("\n--- 4. 이미지 순차 처리 시작 ---")
    processed_count = 0
    error_count = 0
    
    # tqdm으로 진행 상황을 보면서 하나씩 처리합니다.
    for img_path, output_path in tqdm(tasks, desc="이미지 처리"):
        result = process_single_image(img_path, output_path)
        
        # process_single_image 함수는 (경로, 상태) 튜플을 반환합니다.
        path, status = result
        
        if status == "success":
            processed_count += 1
        else:
            error_count += 1
            # 오류가 발생한 파일과 원인을 출력합니다.
            print(f"\n[오류] 파일 처리 실패: {os.path.basename(path)}, 원인: {status}")

    print(f"\n✅ 전처리 완료: 성공 {processed_count}개, 실패 {error_count}개")
else:
    print("\n✅ 처리할 새 이미지가 없어 전처리 작업을 건너뜁니다.")


--- 4. 이미지 순차 처리 시작 ---


이미지 처리: 100%|██████████| 15680/15680 [04:36<00:00, 56.77it/s]


✅ 전처리 완료: 성공 15680개, 실패 0개
